In [1]:
import climlab

In [3]:
def run_column(atm_input, sh_input, gcm_p, co2_lev=3e-4, insol=175.2, alb=0.54,surf_forc=0):
    
    state = climlab.column_state(lev=gcm_p, num_lat=1, water_depth=1.)
    model = climlab.TimeDependentProcess(state=state)
    
    conv = climlab.convection.ConvectiveAdjustment(state=state, adj_lapse_rate='MALR')
    abs_vmr = climlab.radiation.radiation.default_absorbers(model.Tatm)
    abs_vmr['CO2']=co2_lev
    rad = climlab.radiation.RRTMG(state=state,
                                 specific_humidity=sh_input,
                                 albedo = alb,
                                 insolation = insol,
                                 absorber_vmr = abs_vmr)
    myforcing = ExternalForcing(state=state)
    myforcing.forcing_tendencies['Tatm']=atm_input
    myforcing.forcing_tendencies['Ts']=surf_forc
    model.add_subprocess('Radiation', rad)
    model.add_subprocess('ConvectiveAdjustment', conv)
    model.add_subprocess('advective forcing', myforcing)
    model.integrate_years(1,False)
    return model

def run_column_fixedRH(atm_input, rh_input, gcm_p, co2_lev=3e-4, insol=175.2, alb=0.54,surf_forc=0):
    
    state = climlab.column_state(lev=gcm_p, num_lat=1, water_depth=1.)
    model = climlab.TimeDependentProcess(state=state)
    
    conv = climlab.convection.ConvectiveAdjustment(state=state, adj_lapse_rate='MALR')
    abs_vmr = climlab.radiation.radiation.default_absorbers(model.Tatm)
    abs_vmr['CO2']=co2_lev
    h2o = climlab.radiation.water_vapor.FixedRelativeHumidity(state=model.state)
    h2o.RH_profile = rh_input
    rad = climlab.radiation.RRTMG(state=state,
                                 specific_humidity=h2o.q,
                                 albedo = alb,
                                 insolation = insol,
                                 absorber_vmr = abs_vmr)
    myforcing = ExternalForcing(state=state)
    myforcing.forcing_tendencies['Tatm']=atm_input
    myforcing.forcing_tendencies['Ts']=surf_forc
    model.add_subprocess('Radiation', rad)
    model.add_subprocess('ConvectiveAdjustment', conv)
    model.add_subprocess('advective forcing', myforcing)
    model.add_subprocess('WaterVapor', h2o)
    model.integrate_years(1,False)
    return model